#  Step 1: Import all libraries required

In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
import urllib
import numpy as np
import pandas as pd
import matplotlib as plt
import sys 
import importlib
import re

# Step 2A : Web Scrapping - Scrap the link of each job listing

- Store all links to a list called 'href'

- To save the scrapping time, remove all duplicated links by using set function

In [3]:
URL = {'Data analyst':"https://hk.indeed.com/jobs?as_and=data+analyst&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=fulltime&st=&as_src=&salary=&radius=0&l=Hong+Kong&fromage=any&limit=10&sort=&psf=advsrch&from=advancedsearch",
       'Data scientist': 'https://hk.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&as_src=&salary=&radius=0&l=Hong+Kong&fromage=any&limit=10&sort=&psf=advsrch&from=advancedsearch',
       }
#'Data engineer': 'https://hk.indeed.com/jobs?q=data%20engineer&l=Hong%20Kong&radius=0&vjk=7fd03b6ed658179e'
href = {}
for category ,URL in URL.items():
    html = requests.get(URL)
    soup = BeautifulSoup(html.text, "html.parser")    
    cnt = 0

    while True:
        # find all job postings in each page
        for i in soup.find_all('h2',class_='title'):        
            try:
                href['https://hk.indeed.com' + i.a['href']] = category

            except:
                np.nan

        # go to next page
        cnt += 1 # number of times that have proceeded to next page so far
        pg_num = int(soup.find(class_='pagination').find_all('a',attrs = {'href':True})[-1]['href'].split('=')[-1])/10
        # to get the page number from url

        if cnt == pg_num:
            next_pg = soup.find(class_='pagination').find_all('a',attrs = {'href':True})[-1]['href']
            URL2 = 'https://hk.indeed.com'+ next_pg
            html = requests.get(URL2)
            soup = BeautifulSoup(html.text,'lxml')
        else:
            break

In [4]:
len(href.keys())


978

# Step 2B : Web Scrapping - Scrap all required data from each job listing

- Related data:
   - Issued Time
   
   - Job title

   - Company name
   
   - Location
   
   - Job type (Full-Time / Part- Time)
   
   - Job description
   
   - Salary
   
   - Star rating

In [5]:
# get title, company name, salary, job requirement, soft skills, technical skills, industry 
title = []
company = []
ftpt= []
location = []
issued_time = []
description = []
salary = []
review = []
review_num = []

cnt = 0
for i in href.keys():
    # go to each link one by one
    indi_html = requests.get(i)
    indi_soup = BeautifulSoup(indi_html.text,'html.parser')
    
    # find job title
    try:
        indi_title = indi_soup.find('h3',class_="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title").text       
    except:
        indi_title = np.nan
    
    # find company name
    try:
        indi_company = indi_soup.find('div',class_="icl-u-lg-mr--sm icl-u-xs-mr--xs").text     
    except:
        indi_company = np.nan
    # find job type    
    try:
        #to find ptft keywords by joining all results as string
        temp = ''.join([str(i) for i in indi_soup.find_all(class_="jobsearch-JobMetadataHeader-iconLabel")])
        if 'Full-time' in temp and 'Part-time' in temp:
            indi_ftpt = 'Full-time / Part-time'
        elif 'Full-time' in temp:
            indi_ftpt = 'Full-time'
        elif 'Part-time' in temp:
            indi_ftpt = 'Part-time'
        elif 'Temporary' in temp:
            indi_ftpt = 'Temporary'
        else:
            indi_ftpt = np.nan      
    except:
        indi_ftpt = np.nan
    
    # find working location
    try: 
        indi_location = indi_soup.find_all('span',class_="jobsearch-JobMetadataHeader-iconLabel")[0].text
    except:
        indi_location = np.nan
    #find the posting time
    try:
        temp_time = indi_soup.find('div',class_="jobsearch-JobMetadataFooter").text
        indi_time = re.findall('Just posted|Today|\d+',temp_time)[-1]
        if indi_time in ['Just posted','Today']:
            indi_time = '0'
    except:
        indi_time = np.nan
    
    try:
        indi_desc = indi_soup.find('div',id='jobDescriptionText').text
    except:
        indi_desc = np.nan
    
    
   # to find the salary by joining all related html as a string
    temp_salary = "".join([str(i) for i in indi_soup.find_all(class_="jobsearch-JobMetadataHeader-itemWithIcon icl-u-textColor--secondary icl-u-xs-mt--xs")])
    salary_str = re.findall('\$.*0',temp_salary)
    if salary_str:
        indi_salary = salary_str[0]
    else:
        indi_salary = np.nan
           
    # find the popularity
    try:
        temp_review = indi_soup.find(class_="icl-Ratings-starsCountWrapper")['aria-label']
        indi_review = re.findall(r"\d[.]*\d*",str(temp_review))[0]
    except:
        indi_review = np.nan
    # find the number of review
    try:
        indi_review_num = indi_soup.find(class_="icl-Ratings-count").text.replace(' reviews','')
    except:
        indi_review_num = np.nan
    
    # Add all results into related list
    title.append(indi_title)
    company.append(indi_company)
    ftpt.append(indi_ftpt)
    location.append(indi_location)
    issued_time.append(indi_time)
    description.append(indi_desc)
    salary.append(indi_salary)
    review.append(indi_review)
    review_num.append(indi_review_num)
    
    #optional: for monitoring the progress
    cnt+=1
    print(cnt)
    
    
      

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


# Step 3: Data Frame Creation

- Use the data above to create a dataframe


In [21]:
# add a new column for loacation comparasion
Country = ['Hong Kong' for i in range(len(list(href.values())))]
df = pd.DataFrame(data={'Issued time':issued_time,'Country':Country,'Job category': list(href.values()),'Job title':title,'Company':company,'Part time/ Full Time': ftpt,'Star review':review,'Number of reviews': review_num,'Location':location,'Job description': description,'Salary':salary, 'Indeed link': list(href.keys())})


In [22]:
pd.set_option('display.max_columns', 10000)
pd.set_option('display.max_rows', 10000)


# Step 4: Data Cleaning

# Remove Duplicates

- sort data in ascending order based on issued time
- Remove duplicated data based on Job Title and Company
- (since some company may repost and their job listing details)

In [23]:
# sort data in ascending order
df['Issued time'] = df['Issued time'].astype(int)
df.sort_values('Issued time', axis = 0, ascending = True, inplace=True)
df = df.drop_duplicates(['Job title','Company'],keep ='first')

# Remove irrelevant data based on Job Title
- Job title to be removed:
  - 'Support'
  - 'Programmer'
  - 'Intern'
  - 'Executive Assistant'
    

In [24]:
removed_ls = ['Support','Programmer','Intern','Executive Assistant']
df = df[~df['Job title'].isin(removed_ls)]

# Time of Posting 

This indicates that wheather this job listing is still available, which could also implies that whether the job is easy to get. 

In [25]:
def issue_time(x):
    x = str(x)
    if ' days ago' in x:
        modified = x.replace('days ago','')
    elif 'day ago' in x:
        modified = x.replace(' day ago','')
    elif 'Today' in x or 'Just posted' in x:
        modified = '0'
    else:
        modified = x
    if '+' in x:
        modified = x.replace('+',"")
    return modified
df['Issued time'] = df['Issued time'].apply(issue_time)

# Number of Reviews

In [26]:
df['Number of reviews'] = df['Number of reviews'].apply(lambda x: int(x.replace(',','')) if ',' in str(x) else x)
#df['Number of reviews'] = df['Number of reviews'].apply(lambda x: re.sub(r',','',x) if ',' in str(x) else x)
#re.sub > replace a pattern with a string
df    

,Issued time,Country,Job category,Job title,Company,Part time/ Full Time,Star review,Number of reviews,Location,Job description,Salary,Indeed link
612,0,Hong Kong,Data analyst,Analyst Programmer/ Senior .NET Developer,Mtel Solutions Limited,Full-time,NaN,NaN,Hong Kong,"We are digital enabler\n\nWe create ideas, dep...",NaN,https://hk.indeed.com/rc/clk?jk=f49711d0e78bfd...
551,0,Hong Kong,Data analyst,Analyst Programmer (Java/ .NET),Speedy Group Corp. Limited,NaN,NaN,NaN,Hong Kong,Speedy Group is one of the leading integrated ...,NaN,https://hk.indeed.com/rc/clk?jk=ab040ca9bd68f0...
106,0,Hong Kong,Data analyst,"Analyst, Fixed Income and OTC Derivatives",BOCI,Full-time,3.8,5,Hong Kong,See Job Description\n\nThe role\n\nThis incumb...,NaN,https://hk.indeed.com/rc/clk?jk=54c8494dab2114...
563,0,Hong Kong,Data analyst,QA Engineer Automated,International Recruitment Group - IRG ltd,Full-time,NaN,NaN,Hong Kong,"Duties\nDesign, Develop, Maintain Automated Te...",NaN,https://hk.indeed.com/rc/clk?jk=51f27a6b2f73ab...
859,0,Hong Kong,Data scientist,Data Scientist (Team Lead),Conners Consulting,NaN,NaN,NaN,"Sheung Wan, Hong Kong Island",Job Highlights*Formulate long term data analys...,NaN,https://hk.indeed.com/company/Conners-Consulti...
528,0,Hong Kong,Data analyst,IT Audit & Risk Assurance(Analyst to Manager)3...,Xponential Personnel Limited,Full-time / Part-time,NaN,NaN,Hong Kong,XP Personnel is a Digital Recruitment Agency t...,NaN,https://hk.indeed.com/rc/clk?jk=70bdf31b92c718...
660,0,Hong Kong,Data analyst,System Analyst / Analyst Programmer / Programm...,Leadership Solutions Limited,Full-time,NaN,NaN,Hong Kong,Founded in Hong Kong since 2003 to servebusine...,NaN,https://hk.indeed.com/rc/clk?jk=12c53b11e9c798...
264,0,Hong Kong,Data analyst,"AVP/Associate/Analyst, Infrastructure (IT Divi...",BOC International Holdings Ltd,Full-time,3.5,4,Hong Kong,As a leading investment bank in China and Hong...,NaN,https://hk.indeed.com/rc/clk?jk=dedc3668844d4f...
33,0,Hong Kong,Data analyst,Business Analyst,KY & Company,NaN,NaN,NaN,Hong Kong,Perform in depth As Is To Be analysis to deliv...,NaN,https://hk.indeed.com/company/KY-&-Company/job...
631,0,Hong Kong,Data analyst,Change Manager -up to 65K 12 months rolling co...,Protiviti Hong Kong,Full-time,NaN,NaN,Hong Kong,Protiviti is a global consulting firm that del...,NaN,https://hk.indeed.com/rc/clk?jk=2d1d202c2a4025...


# Step 5: Feature Engineering


# Salary

- Objective: - To find out the average salary of Data Analyst and Data Scientist 
- Find Salary from Job Title and 'Salary' Column
- if 'Salary' column is not np.nan, use the data as Salary
- If it is np.nan, use the data from Job Title


# Step A : Find the Salary from title
 
Step A: Find salary from title
- Create a list of min and max salary

Step B: Create min and max salary column
- use the data from step A


In [27]:
def min_max_salary(x):
    value = re.findall('((\d+)(k|K|m|M)*(\d*\.\d*)*(k|K|m|M))|((\d+,d+)-(d*,*d*))|(\d+,\d+)-*(\d*,*\d*)',x)     
    if len(value) == 0:
        min_result = np.nan
        max_result = np.nan
        
    elif len(value) >= 1:
        temp_result = []
        final_result = []
        
        # to remove empty string
        for tpl in value:
            indi_tpl = list(filter(lambda x: x !='', list(tpl)))
            temp_result.append(indi_tpl)
            
        for numls in temp_result:
            # to convert string into number
            if any(j in ['K','k'] for j in numls[0]):
                num = numls[0].replace('K','')
                num = num.replace('k','')
                num = float(num) * 1000
            elif any(j in ['M','m'] for j in numls[0]):
                num = numls[0].replace('M','')
                num = num.replace('m','')
                num = float(num) * 1000000
            elif ',' in numls[0]:
                num = float(''.join(numls[0].split(',')))
            else:
                num = np.nan
            
            final_result.append(num)
        if len(value) == 1:
            min_result, max_result = final_result[0], final_result[0]
        else:
            min_result = final_result[0]
            max_result = final_result[1]
    return [min_result, max_result]


df['Title salary'] = df['Job title'].apply(min_max_salary)
df['Min title salary'] = df['Title salary'].apply(lambda x: x[0])
df['Max title salary'] = df['Title salary'].apply(lambda x: x[1])
            
       

# To showcase the how the regex works:

In [28]:
x = ['System Analyst / Analyst Programmer (Java/.Net) Up to 45K',
     'Analyst Programmer - C#, ASP.NET, MVC, MS SQL | 25K - 35K',
     'Frontend/ Backend Engineer (UP to 45k)',
     'Software Engineer (backend) | Banking & Finance | HKD 40,000 - HKD 50,000',
     'System Analyst | Banking | 40K - 45K',
     'Senior System Analyst | Banking | 45K - 55K',
     'Voice Engineer - up to 50K',
     'Systems Analyst (.NET) - Life Insurance - 42k - Contract',
     'DevOps Engineer - 60-80k ~ Innovative Company!',
     'Devops Engineer - 30k - Contract - MNC',
     'Data Science Analyst (Listed Group) (5 days work, No Sat) (18K-23K)',
     'Insurance Business Analyst-upto 65K 12 months rolling contract',
     'Frontend/ Backend Engineer (UP to 45k)',
     'Insurance Business Analyst-up to $65K',
     'Performance Analyst- up to $60K',
     'Voice Engineer - up to 50K',
     'Investment Manager, A MNC Entreprise Basic Salary: 40K-60K',
     'Analyst Programmer, Web & Mobile App ($30K)',
     'Senior Data Engineer, Machine Learning (HKD 800K - 1.2M)',
     'Senior Software Engineer (Java/TDD) ~ up to 80K',
     'Investment Analyst (in-house) - 30k',
     'Senior Data Management Analyst - Reinsurance - Up to 36k',
     'Multiple Openings* Assistant Data Manager (Finance; Up to 45k)',
     'Analyst Programmer (BI/ETL) (HKD 25-30K)',
     'Data Management Analyst (30-40K)',
     '(Senior) Business Analyst (banking) - 20- 40k',
     'QA / Software Tester 20 ‚Äì 25K',
     'Programmer / Analyst Programmer (Java) 16 ‚Äì 28K',
     'Business Analyst (Insurance) - 25-45k',
     'Senior Financial Analyst - Manufacturing (Up to 38k)',
     'Business Analyst/ Tester (Banking/ IT)- (20-30k)',
     'Manager, Data Scientist (FMCG / Big Data / Up to 60K)']
     
for i in x:
    #temp = i.replace(' ','')
    #value = re.findall('((\d+)-*(\d+)*(k|K|m|M))|((\d+,d+)-(d*,*d*))|[hkd|HKD]\s*(\d+,\d+)-*[hkd|HKD]*(\d*,*\d*)',i)
    value = re.findall('((\d+)(k|K|m|M)*(\d*\.\d*)*(k|K|m|M))|((\d+,d+)-(d*,*d*))|(\d+,\d+)-*(\d*,*\d*)',i)
    #value = re.findall('((\d+)(\d+\d*)*(k|K|m|M))|((\d+,d+)-(d*,*d*))|(\d+,\d+)-*(\d*,*\d*)',i)
    #value = re.findall('((\d+)-*(\d+)*(k|K|m|M))|((\d+,d+)-(d*,*d*))|(\d+,\d+)-*(\d*,*\d*)',temp)
    print('\n',i)
    print(value)




 System Analyst / Analyst Programmer (Java/.Net) Up to 45K
[('45K', '45', '', '', 'K', '', '', '', '', '')]

 Analyst Programmer - C#, ASP.NET, MVC, MS SQL | 25K - 35K
[('25K', '25', '', '', 'K', '', '', '', '', ''), ('35K', '35', '', '', 'K', '', '', '', '', '')]

 Frontend/ Backend Engineer (UP to 45k)
[('45k', '45', '', '', 'k', '', '', '', '', '')]

 Software Engineer (backend) | Banking & Finance | HKD 40,000 - HKD 50,000
[('', '', '', '', '', '', '', '', '40,000', ''), ('', '', '', '', '', '', '', '', '50,000', '')]

 System Analyst | Banking | 40K - 45K
[('40K', '40', '', '', 'K', '', '', '', '', ''), ('45K', '45', '', '', 'K', '', '', '', '', '')]

 Senior System Analyst | Banking | 45K - 55K
[('45K', '45', '', '', 'K', '', '', '', '', ''), ('55K', '55', '', '', 'K', '', '', '', '', '')]

 Voice Engineer - up to 50K
[('50K', '50', '', '', 'K', '', '', '', '', '')]

 Systems Analyst (.NET) - Life Insurance - 42k - Contract
[('42k', '42', '', '', 'k', '', '', '', '', '')]

 DevO

# Step B: Find the max and min Salary from 'Salary' column

In [29]:
def min_max_salary_column(x):
    try:
        salary_str = str(x).replace('$','').replace(',','')

        if '-' in salary_str:
            numls = salary_str.split('-')
            min_salary = int(numls[0])
            max_salary = int(numls[1])
        else:
            min_salary, max_salary = int(salary_str),int(salary_str)
    except:
        min_salary, max_salary = np.nan, np.nan
    
    return [min_salary,max_salary]

df['Salary'] =  df['Salary'].apply(min_max_salary_column) 
df['Min header salary'] = df['Salary'].apply(lambda x: x[0])
df['Max header salary'] = df['Salary'].apply(lambda x: x[1])


# Final Step: Finalised Salary Data

- Min, Max Salary
- Average Salary
- Salary Group

In [30]:
df['Min salary'] = df['Min header salary'].fillna(df['Min title salary'])
df['Max salary'] = df['Max header salary'].fillna(df['Max title salary'])
df['Average salary'] = df.loc[:,'Min salary':'Max salary'].mean(axis=1)

def salary_range(x):
    if x < 10000:
        group = 'Below 10000'
    elif 10000 <= x < 20000:
        group = '10000-19999'
    elif 20000 <= x < 30000:
        group = '20000-29999'
    elif 30000 <= x < 40000:
        group = '30000-39999'
    elif 40000 <= x < 50000:
        group = '40000-49999'
    elif 50000 <= x < 60000:
        group = '50000-59999'
    elif 60000 <= x < 70000:
        group = '60000-69999'
    elif x>= 70000:
        group = '70000 or above'
    else:
        group = 'Not Mentioned'
    return group
    
df['Salary group'] = df['Average salary'].apply(salary_range)    
df = df.drop(['Salary','Title salary','Min header salary','Max header salary','Min title salary','Max title salary'],axis=1)

df

,Issued time,Country,Job category,Job title,Company,Part time/ Full Time,Star review,Number of reviews,Location,Job description,Indeed link,Min salary,Max salary,Average salary,Salary group
612,0,Hong Kong,Data analyst,Analyst Programmer/ Senior .NET Developer,Mtel Solutions Limited,Full-time,NaN,NaN,Hong Kong,"We are digital enabler\n\nWe create ideas, dep...",https://hk.indeed.com/rc/clk?jk=f49711d0e78bfd...,NaN,NaN,NaN,Not Mentioned
551,0,Hong Kong,Data analyst,Analyst Programmer (Java/ .NET),Speedy Group Corp. Limited,NaN,NaN,NaN,Hong Kong,Speedy Group is one of the leading integrated ...,https://hk.indeed.com/rc/clk?jk=ab040ca9bd68f0...,NaN,NaN,NaN,Not Mentioned
106,0,Hong Kong,Data analyst,"Analyst, Fixed Income and OTC Derivatives",BOCI,Full-time,3.8,5,Hong Kong,See Job Description\n\nThe role\n\nThis incumb...,https://hk.indeed.com/rc/clk?jk=54c8494dab2114...,NaN,NaN,NaN,Not Mentioned
563,0,Hong Kong,Data analyst,QA Engineer Automated,International Recruitment Group - IRG ltd,Full-time,NaN,NaN,Hong Kong,"Duties\nDesign, Develop, Maintain Automated Te...",https://hk.indeed.com/rc/clk?jk=51f27a6b2f73ab...,NaN,NaN,NaN,Not Mentioned
859,0,Hong Kong,Data scientist,Data Scientist (Team Lead),Conners Consulting,NaN,NaN,NaN,"Sheung Wan, Hong Kong Island",Job Highlights*Formulate long term data analys...,https://hk.indeed.com/company/Conners-Consulti...,NaN,NaN,NaN,Not Mentioned
528,0,Hong Kong,Data analyst,IT Audit & Risk Assurance(Analyst to Manager)3...,Xponential Personnel Limited,Full-time / Part-time,NaN,NaN,Hong Kong,XP Personnel is a Digital Recruitment Agency t...,https://hk.indeed.com/rc/clk?jk=70bdf31b92c718...,60000.0,60000.0,60000.0,60000-69999
660,0,Hong Kong,Data analyst,System Analyst / Analyst Programmer / Programm...,Leadership Solutions Limited,Full-time,NaN,NaN,Hong Kong,Founded in Hong Kong since 2003 to servebusine...,https://hk.indeed.com/rc/clk?jk=12c53b11e9c798...,NaN,NaN,NaN,Not Mentioned
264,0,Hong Kong,Data analyst,"AVP/Associate/Analyst, Infrastructure (IT Divi...",BOC International Holdings Ltd,Full-time,3.5,4,Hong Kong,As a leading investment bank in China and Hong...,https://hk.indeed.com/rc/clk?jk=dedc3668844d4f...,NaN,NaN,NaN,Not Mentioned
33,0,Hong Kong,Data analyst,Business Analyst,KY & Company,NaN,NaN,NaN,Hong Kong,Perform in depth As Is To Be analysis to deliv...,https://hk.indeed.com/company/KY-&-Company/job...,NaN,NaN,NaN,Not Mentioned
631,0,Hong Kong,Data analyst,Change Manager -up to 65K 12 months rolling co...,Protiviti Hong Kong,Full-time,NaN,NaN,Hong Kong,Protiviti is a global consulting firm that del...,https://hk.indeed.com/rc/clk?jk=2d1d202c2a4025...,65000.0,65000.0,65000.0,60000-69999



# Education Level (Minimum and Maximum Requirement)

to see what education level can master the job

In [31]:
def min_edu_lv(x):
    x = str(x)
    if 'Diploma' in x:
        edu_lv = 'Diploma'
        
    elif ' hd ' in x.lower() or 'higher diploma' in x.lower():
        edu_lv = 'Higher Diploma'
    
    elif 'bachelor' in x.lower() or 'university' in x.lower() or 'degree holder' in x.lower() or 'Degree' in x or 'Bsc' in x:
        edu_lv = 'Bachelor'
        
    elif "master's" in x.lower() or 'MBA' in x or 'Msc' in x or 'Master' in x:
        edu_lv = "Master"
        
    elif 'phd' in x.lower() or 'doctorate' in x.lower():
        edu_lv = 'PhD'
        
    else:
        edu_lv = 'Not specified'

    return edu_lv

def max_edu_lv(x):
    x = str(x)
    if 'phd' in x.lower() or 'doctorate' in x.lower():
        edu_lv = 'PhD'
        
    elif "master's" in x.lower() or 'MBA' in x or 'Msc' in x or 'Master' in x:
        edu_lv = "Master"
    
    elif 'bachelor' in x.lower() or 'university' in x.lower() or 'degree holder' in x.lower() or 'Degree' in x:
        edu_lv = 'Bachelor'
        
    elif ' hd ' in x.lower() or 'higher diploma' in x.lower():
        edu_lv = 'Higher Diploma'
        
    elif 'Diploma' in x:
        edu_lv = 'Diploma'
    else:
        edu_lv = 'Not specified'

    return edu_lv
        

df['Minimum education level'] = df['Job description'].apply(min_edu_lv)
df['Maximum education level'] = df['Job description'].apply(max_edu_lv)    

    

# Skill Set Required
- Create a dictionary to store all skill set required
    - Key : Column Name of each skill
    - Value: key words for matching string

In [32]:
skill_ls = {'Python': ['Python','Python'.lower()], 
             'Machine Learning':['Machine Learning'.lower(), 'Machine Learning'],
             'JavaScript': ['JavaSript'.lower(),'JavaScript'],
             'R':[' R ','R,'],
             'Java':['Java ','Java'.lower(),' Java '],
             'IoT':['IoT'],
             'Excel':['Microsoft Excel','Excel',' Excel '], 
             'Power BI':['BI','Power BI'],
             'English':['English','english'],
             'Chinese':['Cantonese','Cantonese'.lower(),'Mandarin','Mandarin'.lower(),'Chinese','Chinese'.lower(),"Putonghua",'putonghua'],
             'SAS':['SAS','sas'.lower()],
             'Oracle':['Oracle','Oracle'.lower()],
             'C++':['C++','c++'],
             'C#':['C#','c#'],
             'C':[' C ',' c '],
             'Qlik':['Qlik','Qlik'.lower()],
             'alteryx':['Alteryx','Altery'],
             'Tableau':['tableau','Tableau'],
             'SPSS':['SPSS','SPSS'.lower()],
             'Powerpoint':['Microsoft Powerpoint','Microsoft Powerpoint'.lower(),'Powerpoint','powerpoint'.lower()],
             'PostgreSQL':['PostgreSQL','PostgreSQL'.lower()],
             'SQL':[' SQL ',' SQL '.lower()],
             'MySQL':['MySQL','MySQL'.lower()]}    

# Summarize skills into two categories
program_ls = []
visual_ls = []
for ls in [skill_ls['Python'],skill_ls['R']]:
    for j in ls:
        program_ls.append(j)
for ls in [skill_ls['Tableau'], skill_ls['Qlik'], skill_ls['Power BI']]:
    for j in ls:
        visual_ls.append(j)
skill_ls['Visualisation tool'] = visual_ls
skill_ls['Programming tool'] = program_ls
# append to column
for key, value in skill_ls.items():
    df[key] = df['Job description'].apply(lambda x: 'Y'  if any(j in x for j in value) else np.nan)









        

# Star Rating / Company Review

In [33]:
def score(x):
    try:
        if x < 1:
            score = 'Less than 1 star'
        elif 1 <= x <2:
            score = '1.0 to 1.99 stars'
        elif 2 <= x < 3:
            score = '2.0 to 2.99 stars'
        elif 3 <= x < 4:
            score = '3.0 to 3.99 stars'
        elif 4 <= x < 5:
            score = '4.0 to 4.99 stars'
        else:
            score = '5.0 stars'
    except:
            score = np.nan
    return score
df['Star group'] = df['Star review'].apply(score)

In [34]:
df

,Issued time,Country,Job category,Job title,Company,Part time/ Full Time,Star review,Number of reviews,Location,Job description,Indeed link,Min salary,Max salary,Average salary,Salary group,Minimum education level,Maximum education level,Python,Machine Learning,JavaScript,R,Java,IoT,Excel,Power BI,English,Chinese,SAS,Oracle,C++,C#,C,Qlik,alteryx,Tableau,SPSS,Powerpoint,PostgreSQL,SQL,MySQL,Visualisation tool,Programming tool,Star group
612,0,Hong Kong,Data analyst,Analyst Programmer/ Senior .NET Developer,Mtel Solutions Limited,Full-time,NaN,NaN,Hong Kong,"We are digital enabler\n\nWe create ideas, dep...",https://hk.indeed.com/rc/clk?jk=f49711d0e78bfd...,NaN,NaN,NaN,Not Mentioned,Diploma,Bachelor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0 stars
551,0,Hong Kong,Data analyst,Analyst Programmer (Java/ .NET),Speedy Group Corp. Limited,NaN,NaN,NaN,Hong Kong,Speedy Group is one of the leading integrated ...,https://hk.indeed.com/rc/clk?jk=ab040ca9bd68f0...,NaN,NaN,NaN,Not Mentioned,Bachelor,Bachelor,NaN,NaN,Y,NaN,Y,NaN,NaN,NaN,Y,Y,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0 stars
106,0,Hong Kong,Data analyst,"Analyst, Fixed Income and OTC Derivatives",BOCI,Full-time,3.8,5,Hong Kong,See Job Description\n\nThe role\n\nThis incumb...,https://hk.indeed.com/rc/clk?jk=54c8494dab2114...,NaN,NaN,NaN,Not Mentioned,Bachelor,Bachelor,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
563,0,Hong Kong,Data analyst,QA Engineer Automated,International Recruitment Group - IRG ltd,Full-time,NaN,NaN,Hong Kong,"Duties\nDesign, Develop, Maintain Automated Te...",https://hk.indeed.com/rc/clk?jk=51f27a6b2f73ab...,NaN,NaN,NaN,Not Mentioned,Not specified,Not specified,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,Y,NaN,NaN,NaN,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0 stars
859,0,Hong Kong,Data scientist,Data Scientist (Team Lead),Conners Consulting,NaN,NaN,NaN,"Sheung Wan, Hong Kong Island",Job Highlights*Formulate long term data analys...,https://hk.indeed.com/company/Conners-Consulti...,NaN,NaN,NaN,Not Mentioned,Bachelor,Bachelor,Y,Y,NaN,Y,NaN,NaN,Y,Y,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,Y,NaN,Y,Y,5.0 stars
528,0,Hong Kong,Data analyst,IT Audit & Risk Assurance(Analyst to Manager)3...,Xponential Personnel Limited,Full-time / Part-time,NaN,NaN,Hong Kong,XP Personnel is a Digital Recruitment Agency t...,https://hk.indeed.com/rc/clk?jk=70bdf31b92c718...,60000.0,60000.0,60000.0,60000-69999,Bachelor,Bachelor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0 stars
660,0,Hong Kong,Data analyst,System Analyst / Analyst Programmer / Programm...,Leadership Solutions Limited,Full-time,NaN,NaN,Hong Kong,Founded in Hong Kong since 2003 to servebusine...,https://hk.indeed.com/rc/clk?jk=12c53b11e9c798...,NaN,NaN,NaN,Not Mentioned,Bachelor,Bachelor,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0 stars
264,0,Hong Kong,Data analyst,"AVP/Associate/Analyst, Infrastructure (IT Divi...",BOC International Holdings Ltd,Full-time,3.5,4,Hong Kong,As a leading investment bank in China and Hong...,https://hk.indeed.com/rc/clk?jk=dedc3668844d4f...,NaN,NaN,NaN,Not Mentioned,Not specified,Not specified,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,0,Hong Kong,Data analyst,Business Analyst,KY & Company,NaN,NaN,NaN,Hong Kong,Perform in depth As Is To Be analysis to deliv...,https://hk.indeed.com/company/KY-&-Company/job...,NaN,NaN,NaN,Not Mentioned,Not specified,Not specified,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0 stars
631,0,Hong Kong,Data analyst,Change Manager -up to 65K 12 months rolling co...,Protiviti Hong Kong,Full-time,NaN,NaN,Hong Kong,Protiviti is a global consulting firm that del...,https://hk.indeed.com/rc/clk?jk=2d1d202c2a4025.

# Step 6: Export dataframe to CSV

In [20]:
df.to_csv('Indeed.csv')